# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/LP/LP2.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are in charge of the supply purchasing of a company that produces two kinds of drugs. 
The drugs contain a specific active agent, which is extracted from two different kinds of raw materials that should be purchased on the market. 

The goal is to maximize the total profit obtained from producing the drugs, which means minimizing purchasing costs for buying the raw materials and minimizing operational costs for producing the drugs while maximizing sales. 

You are given a budget for purchasing raw materials and operating the production process of the drugs which cannot be exceeded. 
Additionally, you need to keep the capacity constraints for the prod

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

Thank you for providing the problem description. I will formulate a mathematical optimization model to address the supply purchasing problem for your company. The model will include parameters, decision variables, the objective function, and the constraints.

Parameters:
- i: index for raw materials (i = 1, 2)
- j: index for drugs (j = 1, 2)
- p_i: purchasing cost per unit of raw material i
- c_j: operational cost per unit of drug j
- s_j: selling price per unit of drug j
- B: total budget for purchasing raw materials and operating the production process
- M_j: maximum production capacity for drug j (in units)
- R_i: maximum storage capacity for raw material i (in units)
- a_ij: amount of active ingredient extracted from one unit of raw material i for drug j
- d_j: required amount of active ingredient per unit of drug j
- v_i: maximum variation in the amount of active ingredient extracted from raw material i (0.5% for i=1, 2% for i=2)

Decision Variables:
- x_i: amount of raw material 

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Here's the Python code using Pyomo for the optimization problem you described:

```python
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Define sets
model.I = RangeSet(1, 2)  # Set of raw materials
model.J = RangeSet(1, 2)  # Set of drugs

# Define parameters (using sample data)
model.p = Param(model.I, initialize={1: 10, 2: 15})  # Purchasing cost per unit of raw material
model.c = Param(model.J, initialize={1: 5, 2: 8})  # Operational cost per unit of drug
model.s = Param(model.J, initialize={1: 50, 2: 60})  # Selling price per unit of drug
model.B = Param(initialize=1000)  # Total budget
model.M = Param(model.J, initialize={1: 100, 2: 80})  # Maximum production capacity for each drug
model.R = Param(model.I, initialize={1: 200, 2: 150})  # Maximum storage capacity for each raw material
model.a = Param(model.I, model.J, initialize={(1, 1): 0.8, (1, 2): 0.6, (2, 1): 0.7, (2, 2): 0.9})  # Amount of active ingredient extracted
model.d = Param(model.J, 

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
from pyomo.environ import *

# Create a Concrete Model
model = ConcreteModel()

# Define sets
model.I = RangeSet(1, 2)  # Set of raw materials
model.J = RangeSet(1, 2)  # Set of drugs

# Define parameters (using sample data)
model.p = Param(model.I, initialize={1: 100, 2: 199.90})  # Purchasing cost per unit of raw material
model.c = Param(model.J, initialize={1: 700, 2: 800})  # Operational cost per unit of drug
model.s = Param(model.J, initialize={1: 6200, 2: 6900})  # Selling price per unit of drug
model.B = Param(initialize=100000)  # Total budget
model.M = Param(model.J, initialize={1: 100, 2: 80})  # Maximum production capacity for each drug
model.R = Param(model.I, initialize={1: 1000, 2: 1000})  # Maximum storage capacity for each raw material
model.a = Param(model.I, model.J, initialize={(1, 1): 0.01, (1, 2): 0.01, (2, 1): 0.02, (2, 2): 0.02})  # Amount of active ingredient extracted
model.d = Param(model.J, initialize={1: 0.5, 2: 0.6})  # Required amount of active ingredient per unit of drug
model.v = Param(model.I, initialize={1: 0.005, 2: 0.02})  # Maximum variation in the amount of active ingredient extracted

# Define decision variables
model.x = Var(model.I, within=NonNegativeReals)  # Amount of raw material to purchase
model.y = Var(model.J, within=NonNegativeReals)  # Amount of drug to produce

# Define objective function
def profit_rule(model):
    return sum(model.s[j] * model.y[j] for j in model.J) - sum(model.p[i] * model.x[i] for i in model.I) - sum(model.c[j] * model.y[j] for j in model.J)
model.profit = Objective(rule=profit_rule, sense=maximize)

# Define constraints
def budget_rule(model):
    return sum(model.p[i] * model.x[i] for i in model.I) + sum(model.c[j] * model.y[j] for j in model.J) <= model.B
model.budget_constraint = Constraint(rule=budget_rule)

def production_capacity_rule(model, j):
    return model.y[j] <= model.M[j]
model.production_capacity_constraint = Constraint(model.J, rule=production_capacity_rule)

def storage_capacity_rule(model, i):
    return model.x[i] <= model.R[i]
model.storage_capacity_constraint = Constraint(model.I, rule=storage_capacity_rule)

def active_ingredient_rule(model, j):
    return sum(model.a[i, j] * (1 - model.v[i]) * model.x[i] for i in model.I) >= model.d[j] * model.y[j]
model.active_ingredient_constraint = Constraint(model.J, rule=active_ingredient_rule)

# Solve the optimization problem
opt = SolverFactory('glpk')
results = opt.solve(model)

# Print the results
print("Optimal solution found with total profit: ", model.profit())
print("Raw material purchases:")
for i in model.I:
    print(f"Raw material {i}: {model.x[i]():.2f} units")
print("Drug production:")
for j in model.J:
    print(f"Drug {j}: {model.y[j]():.2f} units")

Optimal solution found with total profit:  86958.51568437305
Raw material purchases:
Raw material 1: 786.18 units
Raw material 2: 0.00 units
Drug production:
Drug 1: 15.65 units
Drug 2: 13.04 units


# 5. Correct The Model Code to Test Mathematical Model (if applicable)